# Data Wrangling and Exploration


In [ ]:
import pandas as pd
import numpy as np
import os
import glob
from PIL import Image
import cv2
import pyCAIR as pycair

from skimage import io, transform, util
import matplotlib.pyplot as plt


1. Functions to perform data augmentation (for self supervised learning)
2. Heap map analysis (after seam carving, generate heat map of masks)

In [ ]:
image_dir = "/Users/Amulya/workspace/springboard/Capstone_Project/data/images"
image_files = glob.glob(os.path.join(image_dir,'*.jpg'))

In [ ]:
# Image Augmentation Functions

def patches(data):
    for i in range(0,50):
        r_i = random.randint(0, width-1)
        c_i = random.randint(0, height-1)

        r = random.randint(0, 50)
        c = random.randint(0, 50)
    
        data[r_i:r_i+r, c_i:c_i+c] =0
    
    return data

def zigzag(data):
    n_random_lines = 80
    width, height = pic.size
    n_pixels = 5

    for i in range(n_random_lines):
        r_i = random.randint(0, width-1)
        for j in range(height//n_pixels):
            shift = random.choice([ -10, -1, 1, -2, 2, 10])
        
            if j%3 == 0:
                data[j*n_pixels : (j+1) * n_pixels, (r_i + shift)  % width] = 0
            elif j%3 == 1: 
                data[j*n_pixels : (j+1) * n_pixels, r_i] = 0
            else:
                data[j*n_pixels : (j+1) * n_pixels, (r_i + shift)  % width] = 0
                
    return data

def straightlines(data):
    width, height = 480, 364
    n_random_lines = 80
    #store_random = []
    r_index = []    
    for j in range(n_random_lines):
        r_i = random.randint(0, width-1)
        r_index.append(r_i)
        data[:, r_i] = 0
    #store_random.append(r_index)
    #image_arrays.append(image_array)
    #return data, store_random
    return data

def flipped_image(data):
    data = np.fliplr(data)
    return data


In [ ]:
train_image_files = []

for i in range(0,len(image_files)):
    image = Image.open(image_files[i])
    image = image.resize((480, 364))
    
    #convert image to numpy array
    data = np.array(image)
    
    # patches
    if i%4 == 0:
        data = patches(data)
        train_image_files.append(data)
        
    elif i%4 == 1:
        data = zigzag(data)
        train_image_files.append(data)
    
    elif i%4 == 2:
        data = straightlines(data)
        train_image_files.append(data)
        
    else:
        data = flipped_image(data)
        train_image_files.append(data)
        
        
        
        

### Heat Map analysis

In [ ]:
#After seam carving , collect the mask and check the seam distribution using heat maps

sm_dir = "/Users/Amulya/workspace/springboard/Capstone_Project/su-net/unet_results/seam_output/masks"
sm_files = glob.glob(os.path.join(sm_dir,'*.jpg'))

#heat map for seam carved mask
a = np.zeros([384, 496])
for f in sm_files:
    b = Image.open(f)
    b = np.array(b)
    #print(b.shape) #(384, 496)
    c = np.add(a, b)
    a = c
print(c)

plt.imshow(c)


In [ ]:
#If the heat map is heavily imbalanced - try to balance it like this -

def pixel_ratios(sm_files):
    # Calculates the ratio of number of seam pixel in the mask split
    for f in sm_files:
        a = Image.open(f)
        w, h = a.size
        a = np.array(a)
        b = 0
        split_sum_list = []
        for i in range(0,4):
            split = a[:, b:int(b+w/4)-1] 
            b = b+int(w/4)
            #print("b is" , b)
            #print(split)
            split_sum_list.append(np.sum(split))
        ratio = split_sum_list/ np.linalg.norm(split_sum_list)
        #print (ratio)
        ratios.append(ratio)
    return ratios

In [ ]:
ratios = []
ratios = pixel_ratios(sm_files)

In [ ]:
split_1 = []
split_2 = []
split_3 = []
split_4 = []
few_seams = []

for i in range(0, len(ratios)):
    if ratios[i][0] > 0.5:
        split_1.append(i)
    elif ratios[i][1] > 0.5:
        split_2.append(i)
    elif ratios[i][2] > 0.5:
        split_3.append(i)
    elif ratios[i][3] > 0.5:
        split_4.append(i)
    else:
        few_seams.append(i)

In [ ]:
#Create a folder for balanced masks
import shutil, os
balanced_mask_dir = "/Users/Amulya/workspace/springboard/Capstone_Project/su-net/unet_results/seam_output/balanced_mask"

for i in range(0, 226):
    f = sm_files[split_4[i]]
    shutil.copy(f, balanced_mask_dir)